In [1]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [2]:
full_labels = pd.read_csv('../input_data/full_datasets/fully_labelled_texts.csv')

In [3]:
stemmer = SnowballStemmer("dutch")
np.random.seed(500)

def create_preprocess_corpus(input_corpus, content_name, label_name, smoking=False, second_experiment=False, skip_rows=[]):
    corpus = input_corpus[['Unnamed: 0', content_name, label_name]]
    corpus = corpus.rename({content_name:'text', label_name: 'label'}, axis=1)
    corpus['processed_text'] = corpus['text'].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    if second_experiment:
        if smoking:
            replace_text = 'Rookt niet'
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Rookt niet')
            corpus['label'] = corpus['label'].str.replace('Rookte', 'Rookt niet')
        else:
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Nee')
        
    corpus = corpus.drop(corpus[corpus.label == '--'].index)
    corpus = corpus.drop(corpus[corpus.label == 'Onbekend'].index)
    corpus = corpus[~corpus.processed_text.str.contains('vertrouwelijk')]
    corpus_backup = corpus.copy()
    return corpus, corpus_backup

def add_processed_text(input_corpus, text_column):
    corpus = input_corpus.copy()
    corpus['processed_text'] = corpus[text_column].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    return corpus

In [4]:
existing_labels = pd.read_csv('full_texts_to_label_full4.csv')

In [5]:
existing_random_labels = pd.read_csv('full_texts_to_label_random1.csv')

In [6]:
existing_random_labels2 = pd.read_csv('full_texts_to_label_random2.csv')

In [7]:
full_labels_no_dup = full_labels[~full_labels.content.isin(existing_random_labels.text)][~full_labels.content.isin(existing_random_labels2.text)][~full_labels.content.isin(existing_labels.text)]

/tmp/ipykernel_4061745/1632720.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  full_labels_no_dup = full_labels[~full_labels.content.isin(existing_random_labels.text)][~full_labels.content.isin(existing_random_labels2.text)][~full_labels.content.isin(existing_labels.text)]


In [8]:
full_labels_no_dup

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [9]:
random_samples = full_labels_no_dup.sample(frac=1).head(501)

In [10]:
random_samples

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
20574,453617,Indicatie:\tAfwijkingen? Verdachte gebieden vo...,Niets gevonden,Niets gevonden,Niets gevonden
27458,532490,Samenvatting: \t4/6 mediale epicondyl fractuur...,Niets gevonden,Niets gevonden,Niets gevonden
16884,435825,Indicatie:\tStatus epiduraal hematoom?\t\tKlin...,Niets gevonden,Niets gevonden,Niets gevonden
48709,622354,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Ja,Nee
4604,349374,[INITIALS][LASTNAME] ...,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
31188,549548,Beloop: \tBeloop: Huisarts van HAP belde.\tSte...,Niets gevonden,Niets gevonden,Niets gevonden
59730,651930,UITSLAG MICROBIOLOGIE\t\t[INITIALS] [LASTNAME]...,Niets gevonden,Niets gevonden,Niets gevonden
32962,559954,UITSLAG MICROBIOLOGIE\t\t[INITIALS] [LASTNAME]...,Niets gevonden,Niets gevonden,Niets gevonden
25760,516529,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Nee,Nee


In [11]:
full_samples = add_processed_text(random_samples, 'content')

In [12]:
full_texts_to_label = full_samples.rename(columns={'content':'text'})[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']]

In [13]:
full_texts_to_label.to_csv('full_texts_to_label_random3.csv')